In [1]:
from flask import Flask, render_template,url_for,request,redirect
import pandas as pd
import scipy.stats
import numpy as np
import os
from os.path import join, dirname, realpath

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

In [3]:
def dataProcessing(inp_arr):
    
    scaler=StandardScaler()
    scaler.fit(inp_arr)
    scaled_features=scaler.transform(inp_arr)
    scaled_df=pd.DataFrame(scaled_features,columns=inp_arr.columns)
    print(scaled_df)
    return scaled_df


In [4]:
def prediction(df):
    #print(df.head)
    predictions = dict()
    predictions["rf"] = rf.predict(df.iloc[:,:])
    predictions["svm"] = svm.predict(df.iloc[:,:])
    predictions["dt"] = dt.predict(df.iloc[:,:])
    predictions["lr"] = lr.predict(df.iloc[:,:])
    predictions["knn"] = knn.predict(df.iloc[:,:])
    #predictions["ann"] = ann.predict(df.iloc[:,:])
    return predictions

In [5]:
import pickle

In [6]:
with open('rf_optimal' , 'rb') as f:
    rf = pickle.load(f)
with open('svm_load1' , 'rb') as f:
    svm = pickle.load(f)
with open('d_tree' , 'rb') as f:
    dt = pickle.load(f)
with open('log_reg' , 'rb') as f:
    lr = pickle.load(f)
# with open('ann' , 'rb') as f:
#     ann = pickle.load(f)
with open('KNN_optimal' , 'rb') as f:
    knn = pickle.load(f)

In [7]:
# for i in range(1,1000):
    
#     print(prediction(scaled_df.iloc[i:i+1,:-1]))

In [42]:
app = Flask("__name__")

# Upload folder
UPLOAD_FOLDER = 'static/files'
app.config['UPLOAD_FOLDER'] =  UPLOAD_FOLDER

@app.route("/",methods=['GET','POST'])
def open():
    return render_template("open.html")

def predict(k):
    
    fault_type = ['IR_7_1','IR_14_1','IR_21_1','BL_7_1','BL_14_1','BL_21_1','OR_7_1','OR_14_1','OR_21_1','NORMAL']
    faults = ['INNER RACE FAULT of 0.007 in','INNER RACE FAULT of 0.014 in','INNER RACE FAULT of 0.021 in','BALL FAULT of 0.007 in','BALL FAULT of 0.014 in','BALL FAULT of 0.021 in','OUTER RACE FAULT of 0.007 in','OUTER RACE FAULT of 0.014 in','OUTER RACE FAULT of 0.021 in','NORMAL']
    dictio = {}
    for keys,values in k.items():
        dictio[keys]=list(faults[values[i]] for i in range(len(values)))
    return dictio
    
@app.route("/<algorithm>_<algo>_<acc>_<auc>")
def algorithms(algorithm,algo,acc,auc):
    cr_link="/static/"+algorithm+"_classification_report.jpg"
    cm_link="/static/"+algorithm+"_confusion_matrix.jpg"
    return render_template("algorithm.html",algorithm=algorithm,cr_link=cr_link,cm_link=cm_link,algo=algo,Accuracy=acc,AUC=auc)
@app.route("/uploadcsv",methods=['GET','POST'])
def csv():
    if request.method=='GET':
        return render_template("index.html")
    if request.method == 'POST':
        uploaded_file = request.files['filename']
        if uploaded_file.filename != '':
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], uploaded_file.filename)
          # set the file path
            uploaded_file.save(file_path)
            print(uploaded_file.filename)
            df = pd.read_csv(file_path,header=None)
            #data = np.array(df)
            dat = dataProcessing(df.iloc[1:,:-3])
            #print(randomf.predict(dat.iloc[:,:]))
            print(prediction(dat))
            k=prediction(dat)
            #print(stats.mode(prediction(dat)))
          # save the filewqe1
            dictio=predict(prediction(dat))
            length=len(dictio["rf"])
            
            
            return render_template("enter.html",n=length,rf=dictio["rf"],svm=dictio["svm"],knn=dictio["knn"],dt=dictio["dt"],lr=dictio["lr"])




    
    

#
if __name__ == "__main__":
    app.run()
    

    

 * Serving Flask app "__name__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Apr/2022 01:21:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2022 01:22:21] "GET /uploadcsv HTTP/1.1" 200 -
